In [24]:
# This file gets OAuth tokens
import requests
import json
import csv
import praw
import re
from praw.models import MoreComments
from tickers_names import *
import yfinance as yf
import nltk
from nltk.corpus import stopwords
import datetime
from datetime import datetime as dt
import time
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\timot\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# MAKE SURE TO PUT PASSWORD AND SECRET KEY IN SEPERATE FILE BEFORE MAKING REPO PUBLIC!!!


CLIENT_ID = "9whZ6oWY9qQBkA"
SECRET_KEY = "v4JApa9eu1WvSTmy2eaoTmRHTQ33bw"

auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_KEY)
data = {'grant_type': 'client_credentials',
        'username': 'CS229project',
        'password': '229229'}
headers = {'User-Agent': 'CS229project/0.0.1'}
base_url = 'https://www.reddit.com/'
res = requests.post(base_url + 'api/v1/access_token', auth=auth, data=data, headers=headers)
# convert response to JSON and pull access_token value
res_json = res.json()
print(res_json)


{'access_token': '-8_N7yCmIp3YayU6sFFCz7L47bVp1_A', 'token_type': 'bearer', 'expires_in': 3600, 'scope': '*'}


In [3]:
reddit = praw.Reddit(
    client_id=CLIENT_ID, 
    client_secret=SECRET_KEY, 
    user_agent='CS229project/0.0.1',
    username='CS229project',
    password='229229')

In [4]:

TOKEN = res_json['access_token']
# add authorization to our headers dictionary
headers = {'User-Agent': 'CS229project/0.0.1', 'Authorization': f'bearer {TOKEN}', }

In [5]:
# general post data from r/wsb hot 
r = requests.get('https://oauth.reddit.com/r/wallstreetbets/hot', headers=headers)
data = json.loads(r.text)['data']['children']

with open('reddit_data.csv', 'w', encoding='utf8') as f:  
    writer = csv.writer(f)
    for dp in data:
        for key, value in dp['data'].items(): 
            writer.writerow([key, value])
print("reddit_data.csv file written") 

reddit_data.csv file written


In [6]:
# while the token is valid (~2 hours) we just add headers=headers to our requests
res = requests.get('https://oauth.reddit.com/api/v1/me', headers=headers).json()
print(res)

{'features': {'mod_service_mute_writes': True, 'promoted_trend_blanks': True, 'show_amp_link': True, 'mweb_in_feed_refresh': {'owner': 'growth', 'variant': 'random', 'experiment_id': 507}, 'is_email_permission_required': False, 'mod_awards': True, 'mweb_xpromo_revamp_v3': {'owner': 'growth', 'variant': 'treatment_1', 'experiment_id': 480}, 'chat_subreddit': True, 'awards_on_streams': True, 'mweb_xpromo_modal_listing_click_daily_dismissible_ios': True, 'modlog_copyright_removal': True, 'do_not_track': True, 'mod_service_mute_reads': True, 'chat_user_settings': True, 'use_pref_account_deployment': True, 'mweb_xpromo_interstitial_comments_ios': True, 'mweb_xpromo_modal_listing_click_daily_dismissible_android': True, 'premium_subscriptions_table': True, 'mweb_xpromo_interstitial_comments_android': True, 'mweb_footer_upsell': {'owner': 'growth', 'variant': 'light_1', 'experiment_id': 497}, 'chat_group_rollout': True, 'resized_styles_images': True, 'spez_modal': True, 'noreferrer_to_noopener

In [102]:
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers).json()

{'features': {'mod_service_mute_writes': True,
  'promoted_trend_blanks': True,
  'show_amp_link': True,
  'is_email_permission_required': False,
  'mod_awards': True,
  'expensive_coins_package': True,
  'mweb_xpromo_revamp_v2': {'owner': 'growth',
   'variant': 'treatment_4',
   'experiment_id': 457},
  'awards_on_streams': True,
  'mweb_xpromo_modal_listing_click_daily_dismissible_ios': True,
  'chat_subreddit': True,
  'modlog_copyright_removal': True,
  'do_not_track': True,
  'mod_service_mute_reads': True,
  'chat_user_settings': True,
  'use_pref_account_deployment': True,
  'mweb_xpromo_interstitial_comments_ios': True,
  'mweb_xpromo_modal_listing_click_daily_dismissible_android': True,
  'premium_subscriptions_table': True,
  'mweb_xpromo_interstitial_comments_android': True,
  'noreferrer_to_noopener': True,
  'mweb_footer_upsell': {'owner': 'growth',
   'variant': 'control_1',
   'experiment_id': 497},
  'chat_group_rollout': True,
  'resized_styles_images': True,
  'spez_

In [28]:
url = "https://www.reddit.com/r/wallstreetbets/comments/nc4z12/daily_discussion_thread_for_may_14_2021/"
comments = reddit.submission(url=url).comments[:50]
i = 0
t, t_dict = convert_data()
t_list = [x.lower() for x in t[1:]]
print(t_list[0:100])
stop_words = set(stopwords.words('english'))
add_words = ('one', 'all', 'fang', 'dd', 'elon')
for word in add_words:
    stop_words.add(word)
# comments.replace_more(limit=0)
with open('reddit_data/reddit_data.csv', 'w', encoding='utf8') as f:  
    writer = csv.writer(f)
    for comment in comments:
        #print(comment.body.lower())
        i += 1
        # extract mentioned stocks
        ticker_list = []
        words = []
        for word in comment.body.lower().split():
            words.append(" ".join(re.findall("[a-zA-Z]+", word)))
        print("####\n", words)
        for word in words:
            if (word in t_list or (len(word) > 1 and word[0] == "$" and word[1:] in t_list)) and (word not in ticker_list) and (word not in stop_words):
                ticker_list.append(word)
            elif word in t_dict and t_dict[word] not in ticker_list and word not in stop_words:
                ticker_list.append(t_dict[word])
        # get percent change, write rows
        next_day = str(dt.fromtimestamp(comment.created_utc + 86400)).split()[0]
        for ticker in ticker_list:
            try:  
                df = yf.Ticker(ticker).history(start=next_day, interval="5d")
                print(df.iloc[1][3], df.iloc[0][0])
                percent_change = df.iloc[1][3] / df.iloc[0][0]
                writer.writerow([comment.body.lower(), ticker, percent_change])
            except:
                pass
print("num comments: ", i)

['aos', 'abt', 'abbv', 'abmd', 'acn', 'atvi', 'adbe', 'aap', 'amd', 'aes', 'afl', 'a', 'apd', 'akam', 'alk', 'alb', 'are', 'alxn', 'algn', 'alle', 'lnt', 'all', 'googl', 'goog', 'mo', 'amzn', 'amcr', 'aee', 'aal', 'aep', 'axp', 'aig', 'amt', 'awk', 'amp', 'abc', 'ame', 'amgn', 'aph', 'adi', 'anss', 'antm', 'aon', 'apa', 'aapl', 'amat', 'aptv', 'adm', 'anet', 'ajg', 'aiz', 't', 'ato', 'adsk', 'adp', 'azo', 'avb', 'avy', 'bkr', 'bll', 'bac', 'bax', 'bdx', 'brk.b', 'bby', 'bio', 'biib', 'blk', 'ba', 'bkng', 'bwa', 'bxp', 'bsx', 'bmy', 'avgo', 'br', 'bf.b', 'chrw', 'cog', 'cdns', 'czr', 'cpb', 'cof', 'cah', 'kmx', 'ccl', 'carr', 'ctlt', 'cat', 'cboe', 'cbre', 'cdw', 'ce', 'cnc', 'cnp', 'cern', 'cf', 'crl', 'schw', 'chtr']
####
 ['me', 'a', 'pltr', 'bull', 'i', 'will', 'never', 'financially', 'recover', 'from', 'the', 'largest', 'bull', 'market', 'run', 'in', 'history']
####
 ['i', 'wonder', 'which', 'one', 'of', 'my', 'stocks', 'elon', 'is', 'going', 'to', 'tank', 'today']
####
 ['short', 

In [163]:
print()

<class 'dict'>


In [36]:
url = "https://www.reddit.com/r/wallstreetbets/comments/nc4z12/daily_discussion_thread_for_may_14_2021/"
comment = reddit.submission(url=url).comments[0]
print(comment.created_utc)

1620988034.0
